In [1]:
''' 
 Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
 Date; 7. 22. 2021
 Title: Linear Regression
 Reference: https://wikidocs.net/59425'''

' \n Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering\n Date; 7. 22. 2021\n Title: Linear Regression\n Reference: https://wikidocs.net/59425'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# One-hot encoding
- 정수 인코딩과 달리 원-핫 인코딩은 분류 문제 모든 클래스 간의 관계를 균등하게 분배
- 모든 클래스에 원-핫 인코딩을 통해 얻은 원-핫 벡터들은 모든 쌍의 유클리드 거리를 구해도 전부 유클리드 거리가 동일
- 원-핫 벡터는 이처럼 각 클래스의 표현 방법이 무작위성

# 소프트맥스 회귀
- 확률의 총 합이 1이 되는 이 아이디어를 다중 클래스 분류 문제에 적용
- 선택지의 개수만큼의 차원을 가지는 벡터
- 해당 벡터가 벡터의 모든 원소의 합이 1이 되도록 원소들의 값을 변환시키는 어떤 함수를 지나게 만들어야 한다.
- 소프트맥스 함수의 출력값은 결국 예측값

# 오차함수: 크로스 엔트로피 함수 사용

In [3]:
torch.manual_seed(1)

In [4]:
z = torch.rand(3, 5, requires_grad = True)
hypothesis = F.softmax(z, dim = 1)

# torch.randint: (행렬안에 들어갈 숫자 범위, (행,열)->shape)

In [5]:
y = torch. randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


# 원-핫 인코딩 만드는 방법

In [6]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

# 순서
- 1. torch.zeros_like(hypothesis)를 통해 모든 원소가 0의 값을 가진 3 × 5 텐서를 만듭니다. 그리고 이 텐서는 y_one_hot에 저장이 된 상태
- 2. 언스퀴즈(Unsqueeze) - 특정 위치에 1인 차원을 추가 => y.unsqueeze(1)를 하면 (3,)의 크기를 가졌던 y 텐서는 (3 × 1) 텐서가 됨
- 3. scatter의 첫번째 인자로 dim=1에 대해서 수행, 세번째 인자에 숫자 1을 넣어주므로서 두번째 인자인 y_unsqeeze(1)이 알려주는 위치에 숫자 1을 넣음
- 4. 연산 뒤에 '_'를 붙이면 In-place Operation (덮어쓰기 연산)임

# 파이토치로 소프트맥스의 비용 함수 구현 (로우레벨)

In [7]:
cost = (y_one_hot * - torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


# 파이토치로 소프트맥스의 비용 함수 구현 (하이레벨)
# 1. F.softmax() + torch.log() = F.log_softmax()

- torch.log(F.softmax(z, dim=1))를 방금 배운 F.log_softmax()로 대체할 수 있다.
=> (y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean()

# 2. F.log_softmax() + F.nll_loss() = F.cross_entropy()
- F.nll_loss()를 사용할 때는 원-핫 벡터를 넣을 필요없이 바로 실제값을 인자로 사용
- nll이란 Negative Log Likelihood의 약자
- =>F.nll_loss(F.log_softmax(z, dim=1), y)

# ※ F.cross_entropy는 비용 함수에 소프트맥스 함수까지 포함하고 있음을 기억하고 있어야 구현 시 혼동하지 않습니다.
- F.cross_entropy(z, y)

# 소프트맥스 회귀 구현

In [16]:
x_train = [[1,2],[2,2],[3,2],[4,2],[5,2],[6,2]] # 6개의 샘플, 각 샘플 당 2개의 특징
y_train = [2,2,0,1,1,1] #3개의 클래스
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6])


In [17]:
Y_one_hot = torch.zeros(6,3)
Y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(Y_one_hot.shape)

torch.Size([6, 3])


In [ ]:
W = torch.zeros((4,3), requires_grad = True)